In [2]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Dict, Set, Tuple
import random

@dataclass
class SimParams:
    # Base mutation rate per base pair per year
    u: float = 1.25e-8
    
    # Number of driver positions in each gene
    n_APC: int = 604    # Number of APC driver positions
    n_TP53: int = 73    # Number of TP53 driver positions
    n_KRAS: int = 20    # Number of KRAS driver positions
    
    # Derived mutation rates per year
    r_APC: float = 604 * 1.25e-8    # APC mutation rate
    r_TP53: float = 73 * 1.25e-8    # TP53 mutation rate
    r_KRAS: float = 20 * 1.25e-8    # KRAS mutation rate
    r_LOH: float = 1.36e-4          # Rate of loss of heterozygosity
    
    # Division rates per year  
    b_APC: float = 0.2     # Division rate for APC-/- crypts
    b_KRAS: float = 0.07   # Division rate for KRAS+ crypts
    b_BOTH: float = 0.27   # Division rate for APC-/-,KRAS+ crypts (b_APC + b_KRAS)
    
    # Time parameters
    dt: float = 0.01      # Time step size (tau)
    t_max: float = 80.0   # Maximum simulation time
    
    # Initial conditions
    N_crypts: int = 10**8  # Number of initial crypts

class State:
    """Represents the genetic state of a crypt"""
    def __init__(self, APC: int, TP53: int, KRAS: int):
        # APC states:
        # 0: Wild type
        # 1: First hit LOH
        # 2: First hit mutation
        # 3: Second hit after LOH or mutation
        # 4: Double mutation
        self.APC = APC   
        
        # TP53 states: similar to APC
        self.TP53 = TP53 
        
        # KRAS states:
        # 0: Wild type
        # 1: Activated
        self.KRAS = KRAS
        
    def __hash__(self):
        return hash((self.APC, self.TP53, self.KRAS))
    
    def __eq__(self, other):
        return (self.APC, self.TP53, self.KRAS) == (other.APC, other.TP53, other.KRAS)
    
    def is_malignant(self):
        """Check if state represents a malignant crypt"""
        # Malignant states: (3,3,1), (3,4,1), (4,3,1), (4,4,1)
        return ((self.APC >= 3) and (self.TP53 >= 3) and (self.KRAS == 1))

def get_transition_rate(source: State, target: State, params: SimParams) -> float:
    """Calculate transition rate between states based on Supplementary Materials"""
    # APC transitions
    if target.APC > source.APC:
        if source.APC == 0:
            # First hit can be LOH or mutation
            return params.r_LOH if target.APC == 1 else params.r_APC
        elif source.APC == 1:
            # After LOH, only mutation possible
            return params.r_APC / 2
        elif source.APC == 2:
            # After mutation, both LOH and mutation possible
            return params.r_LOH / 2 if target.APC == 3 else params.r_APC / 2
            
    # TP53 transitions (similar to APC)
    if target.TP53 > source.TP53:
        if source.TP53 == 0:
            return params.r_LOH if target.TP53 == 1 else params.r_TP53
        elif source.TP53 == 1:
            return params.r_TP53 / 2
        elif source.TP53 == 2:
            return params.r_LOH / 2 if target.TP53 == 3 else params.r_TP53 / 2
            
    # KRAS activation
    if target.KRAS > source.KRAS:
        return params.r_KRAS
        
    return 0.0

def get_division_rate(state: State, params: SimParams) -> float:
    """Get division rate for a given state based on Supplementary Materials"""
    if state.APC >= 3:  # APC-/-
        if state.KRAS == 1:  # Also KRAS+
            return params.b_BOTH
        return params.b_APC
    elif state.KRAS == 1:  # Only KRAS+
        return params.b_KRAS
    return 0.0

def get_neighbors(state: State) -> Set[State]:
    """Get all possible next states from current state"""
    neighbors = set()
    
    # APC transitions
    if state.APC == 0:
        neighbors.add(State(1, state.TP53, state.KRAS))  # LOH
        neighbors.add(State(2, state.TP53, state.KRAS))  # Mutation
    elif state.APC == 1:
        neighbors.add(State(3, state.TP53, state.KRAS))  # Mutation after LOH
    elif state.APC == 2:
        neighbors.add(State(3, state.TP53, state.KRAS))  # LOH after mutation
        neighbors.add(State(4, state.TP53, state.KRAS))  # Second mutation
        
    # TP53 transitions
    if state.TP53 == 0:
        neighbors.add(State(state.APC, 1, state.KRAS))  # LOH
        neighbors.add(State(state.APC, 2, state.KRAS))  # Mutation
    elif state.TP53 == 1:
        neighbors.add(State(state.APC, 3, state.KRAS))  # Mutation after LOH
    elif state.TP53 == 2:
        neighbors.add(State(state.APC, 3, state.KRAS))  # LOH after mutation
        neighbors.add(State(state.APC, 4, state.KRAS))  # Second mutation
        
    # KRAS activation
    if state.KRAS == 0:
        neighbors.add(State(state.APC, state.TP53, 1))
        
    return neighbors

def tau_leaping_simulation(params: SimParams, n_runs: int = 1000) -> np.ndarray:
    """Run multiple tau-leaping simulations following supplementary materials"""
    time_points = np.arange(0, params.t_max + params.dt, params.dt)
    malignant_counts = np.zeros(len(time_points))
    
    for run in range(n_runs):
        if run % 100 == 0:
            print(f"Run {run}/{n_runs}")
            
        # Initialize population dictionary
        population = {State(0,0,0): params.N_crypts}
        had_malignant = False
        
        for t_idx, t in enumerate(time_points):
            if had_malignant:
                malignant_counts[t_idx:] += 1
                break
                
            # Process each populated state
            new_events = {}
            for state, count in list(population.items()):
                if count == 0:
                    continue
                    
                # Check for division events (negative binomial distribution)
                division_rate = get_division_rate(state, params)
                if division_rate > 0:
                    p = np.exp(-division_rate * params.dt)
                    n_divisions = np.random.negative_binomial(count, p)
                    if n_divisions > 0:
                        new_events[state] = new_events.get(state, 0) + n_divisions
                
                # Check for transitions to neighbor states
                for neighbor in get_neighbors(state):
                    rate = get_transition_rate(state, neighbor, params)
                    if rate > 0:
                        # Use Poisson approximation for small rates
                        if rate * params.dt < 0.01:
                            n_transitions = np.random.poisson(rate * params.dt * count)
                        else:
                            # Use binomial for larger rates
                            p = 1 - np.exp(-rate * params.dt)
                            n_transitions = np.random.binomial(count, p)
                            
                        if n_transitions > 0:
                            new_events[state] = new_events.get(state, 0) - n_transitions
                            new_events[neighbor] = new_events.get(neighbor, 0) + n_transitions
                            
                            if neighbor.is_malignant():
                                had_malignant = True
                                break
            
            # Update population
            for state, delta in new_events.items():
                population[state] = population.get(state, 0) + delta
                if population[state] < 0:  # Sanity check
                    population[state] = 0
                    
            if had_malignant:
                malignant_counts[t_idx:] += 1
                break
    
    return malignant_counts / n_runs

def plot_results(time_points: np.ndarray, probabilities: np.ndarray):
    """Plot the probability of malignancy over time"""
    plt.figure(figsize=(10, 6))
    plt.semilogy(time_points, probabilities, 'b-', label='Tau-leaping simulation')
    plt.xlabel('Age (years)')
    plt.ylabel('Probability of malignancy')
    plt.title('Probability of Colorectal Cancer Development')
    plt.grid(True)
    plt.legend()
    plt.show()

# Run simulation
#if __name__ == "__main__":
#    params = SimParams()
#    time_points = np.arange(0, params.t_max + params.dt, params.dt)
#    probabilities = tau_leaping_simulation(params, n_runs=10000)
#    plot_results(time_points, probabilities)

# params = SimParams(
#     dt = 0.1,           # 5-year time steps
#     t_max = 80.0,       # Full lifetime
#     N_crypts = 10**8    # Optionally reduce initial crypts
# )

# # Run with fewer simulations
# time_points = np.arange(0, params.t_max + params.dt, params.dt)
# probabilities = tau_leaping_simulation(params, n_runs=8000000)
# plot_results(time_points, probabilities)

In [3]:
from multiprocessing import Pool, cpu_count

def parallel_simulation(task_params):
    """
    Wrapper for tau-leaping simulation to run in parallel.
    """
    params, n_runs = task_params
    return tau_leaping_simulation(params, n_runs)

def run_parallel_simulations(params: SimParams, n_runs: int, n_processes: int = None):
    """
    Split simulation into multiple processes to utilize multi-core CPU.
    """
    if n_processes is None:
        n_processes = cpu_count()  # Use all available CPU cores by default
    
    # Divide the total runs among processes
    runs_per_process = n_runs // n_processes
    task_params = [(params, runs_per_process) for _ in range(n_processes)]
    
    print(f"Running {n_runs} simulations across {n_processes} processes...")
    
    with Pool(n_processes) as pool:
        results = pool.map(parallel_simulation, task_params)
    
    # Combine results from all processes
    combined_probabilities = np.sum(results, axis=0) / n_processes
    return combined_probabilities


In [ ]:
# 保留您的原始参数
params = SimParams(
    dt=0.1,  # 时间步长
    t_max=80.0,  # 最大时间
    N_crypts=10**8  # 初始细胞数
)

# 时间点数组
time_points = np.arange(0, params.t_max + params.dt, params.dt)

# 使用并行运行 800 万次模拟
probabilities = run_parallel_simulations(params, n_runs=8000000)

# 绘制结果
plot_results(time_points, probabilities)

Running 8000000 simulations across 10 processes...


Process SpawnPoolWorker-26:
Process SpawnPoolWorker-24:
Process SpawnPoolWorker-27:
Process SpawnPoolWorker-23:
Process SpawnPoolWorker-28:
Process SpawnPoolWorker-21:
Process SpawnPoolWorker-29:
Process SpawnPoolWorker-22:
Process SpawnPoolWorker-25:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/li